In [181]:
import pandas as pd
import folium
from folium import plugins, features
import numpy as np
from load_month import TripsLoader
import branca.colormap as cm

In [13]:
loader = TripsLoader()
coords = loader.get_coords()

In [47]:
regs = np.loadtxt('data/regs.txt', delimiter=',', dtype=np.int)
print(regs)

[1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174
 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224
 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278
 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333
 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385
 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441
 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068
 2069 2118 2119 2168]


In [8]:
l_parse_dates = [u'tpep_pickup_datetime']
l_index_col = ['tpep_pickup_datetime']

df = pd.read_csv('data/ytd_may15.csv', sep=';', parse_dates=l_parse_dates, index_col=l_index_col)
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2015-06-01 00:00:00,40,32,7,17,60,129,153,271,294,70,...,8,17,10,639,437,230,15,271,90,123
2015-06-01 01:00:00,17,20,5,13,38,90,111,163,246,28,...,6,7,22,547,490,212,21,237,124,119
2015-06-01 02:00:00,5,5,0,8,25,47,97,139,207,25,...,6,11,10,234,215,95,69,253,105,112
2015-06-01 03:00:00,5,6,0,6,27,26,68,77,208,29,...,13,0,2,84,10,17,6,78,3,94
2015-06-01 04:00:00,8,16,2,6,21,39,53,81,109,8,...,20,1,0,9,4,14,5,62,1,1


In [18]:
loc = [(coords[1] + coords[3])/2, (coords[0] + coords[2])/2]
print(loc)

[40.705825, -73.9778]


In [25]:
folium_map = folium.Map(location=loc,
                        zoom_start=11,
                        tiles="CartoDB dark_matter")

folium_map

In [63]:
reg = pd.read_csv('regions.csv', sep=';', index_col=['region'])
reg.head()

,west,east,south,north
region,,,,
1,-74.25559,-74.244478,40.496120,40.504508
2,-74.25559,-74.244478,40.504508,40.512896
3,-74.25559,-74.244478,40.512896,40.521285
4,-74.25559,-74.244478,40.521285,40.529673
5,-74.25559,-74.244478,40.529673,40.538061


In [206]:
use_reg = reg.loc[regs, :]
use_reg['lat'] = (reg.south + reg.north) / 2 
use_reg['lon'] = (reg.west + reg.east) / 2
use_reg = use_reg[['lat', 'lon']]

In [207]:
folium_map = folium.Map(location=loc,
                        zoom_start=11,
                        tiles="CartoDB dark_matter")
data = []
for i in range(6):
    data.append(np.hstack((use_reg.values, df.iloc[i, :].values.reshape(-1, 1))).tolist())
plugins.HeatMapWithTime(data, radius = 12).add_to(folium_map)
folium_map

In [199]:
folium_map = folium.Map(location=loc,
                        zoom_start=11,
                        tiles="CartoDB dark_matter")

In [202]:


data = reg.loc[regs, :].copy()
trips = df.iloc[1, :].values
data['trips'] = trips
data['color'] = np.log(data.trips+1)
data.head()

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=0, vmax=data.color.max()
)
for ind, row in data.iterrows():
    color = linear(row.color)
    popup_text = "Region number:{}<br> Trips from this area: {}"
    popup_text = popup_text.format(ind, int(row.trips))
    features.RectangleMarker(bounds=(row.north, row.west,
                                     row.south, row.east),
                             popup=popup_text,
                             fill_opacity=0.25,
                             fill_color = color,
                             color = 'black',
                            ).add_to(folium_map)

folium_map

In [198]:
folium_map